- Not Only SQL(NoSQL)
- 정해진 규칙이 없다. (Schema X)
- JOIN 구문 사용 불가
- key / value 이루어진 데이터를 담는다
- Database - Collection(table) - Document(row)

### db.user.find( {condition} , {column list} ) 

- db.user.find() - select * from user

- db.user.find({}, { user_id : 1, _id : 0 })- select user_id from user  # 1은 visible, 2는 invisible

- db.user.find({gender : 'f'}, {} ) - select * from user where XXXXXX

- db.user.find({user_id : 'supreme-ys', gender : 'm'}, { }) - select * from user where user_id = xxx and gender = xxx

- db.user.find( { $or :  [ {user_id : 'supreme-ys'} , {gender : 'm'} ] } , { }) - select * from user where user_id = xxx or gender = xxx

### 만약 수정하고 싶다면?
- db.collection_name.updateOne({condition}, {set})
- db.collection_name.updateMany()
- 예시)
- update user
  set gender = 'f'
  where user_id = 'supreme-ys'

- db.user.updateOne({ user_id : 'supreme-ys'}, {$set : {gender: 'f'} })

### 만약 삭제하고 싶다면?
- db.user.removeOne({where})
- db.user.removeMany({where})
- DBMS) delete from table
where condition ;

### 만약 입력하고 싶다면?
- db.collection_name.insertOne({key : value})
- db.collection_name.insertMany([{key : value}]) 리스트안에 딕셔너리 형태를 만들어서 넣는다.

In [3]:
# pip install pymongo

In [36]:
import pymongo as mongo

- 1. connection mongodb

In [37]:
# local
conn = mongo.MongoClient()

# remote
# conn = mongo.MongoClient(server, 27017)

- 2. Database create

In [38]:
articleDB = conn.articleDB
print(articleDB)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'articleDB')


In [39]:
articleCollection = articleDB['article']
print(articleCollection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'articleDB'), 'article')


- 3. Document Insert
- insert_one()(pymongo 방식) - insertOne()(mongo shell 방식)
- insert_many()(pymongo 방식) - insertMany()(mongo shell 방식)

In [40]:
doc01 = {'author' : 'yssim',
         'subject' : 'MongoDB',
         'content' : 'SO EASY', 
         'tags' : ['mongodb', 'python', 'pymongo'],
         'view' : 50}

In [41]:
articleCollection.insert_one(doc01)

- 4. Document Find
- find_one()
- find() - 전체를 가져올 때

In [42]:
articleCollection.find_one()

{'_id': ObjectId('6046fab13435c03992181fa9'),
 'author': 'yssim',
 'subject': 'MongoDB',
 'content': 'SO EASY',
 'tags': ['mongodb', 'python', 'pymongo'],
 'view': 50}

In [43]:
doc02 = {'author' : 'youngseok',
         'subject' : 'MongoDataBase',
         'content' : 'SO DIFFICULT', 
         'tags' : ['mongo', 'python', 'pymongo'],
         'view' : 100}

In [44]:
articleCollection.insert_one(doc02)

In [45]:
articleCollection.find_one() # 가장 먼저 만나는 행을 가져오기 때문에 doc01이 출력이 된다.

{'_id': ObjectId('6046fab13435c03992181fa9'),
 'author': 'yssim',
 'subject': 'MongoDB',
 'content': 'SO EASY',
 'tags': ['mongodb', 'python', 'pymongo'],
 'view': 50}

In [46]:
articleCollection.find_one({'author' : 'youngseok'}) # 조건을 넣어서 특정 데이터를 추출한다.

{'_id': ObjectId('6046fb763435c03992181faa'),
 'author': 'youngseok',
 'subject': 'MongoDataBase',
 'content': 'SO DIFFICULT',
 'tags': ['mongo', 'python', 'pymongo'],
 'view': 100}

In [47]:
cursor = articleCollection.find({'author' : 'youngseok'})
for doc in cursor:
    print(doc) # 전체를 가져오려면 단순히 find를 쓰는 것이 아니라 loop를 돌려서 가져와야한다. 

{'_id': ObjectId('6046fb763435c03992181faa'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 100}
{'_id': ObjectId('604701de3435c03992181fad'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 100}


In [52]:
doc03 = {'author' : 'youngseok',
         'subject' : 'MongoDataBase',
         'content' : 'SO DIFFICULT', 
         'tags' : ['mongo', 'python', 'pymongo'],
         'view' : 200}

In [53]:
articleCollection.insert_one(doc03)

In [74]:
# author가 youngseok이고view가 200이상인 데이터를 추출한다면????

cursor = articleCollection.find( {'author' : 'youngseok', 'view' : {'$gte' : 200} } )
for doc in cursor:
    print(doc)

{'_id': ObjectId('6047041b3435c03992181faf'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 200}


In [77]:
# 위 조건에서 원하는 컬럼만 가져온다면?
cursor = articleCollection.find( {'author' : 'youngseok', 'view' : {'$gte' : 200} } , {'author' : 1, '_id':0})
for doc in cursor:
    print(doc)

{'author': 'youngseok'}


In [83]:
# 미션!
# author 가 yssim인 것에 대한 view를 150으로 바꿔라
articleCollection.update_one( {'author':'yssim'}, {'$set' : {'view' : 150} } )
for doc in articleCollection.find():
    print(doc)

{'_id': ObjectId('6046fab13435c03992181fa9'), 'author': 'yssim', 'subject': 'MongoDB', 'content': 'SO EASY', 'tags': ['mongodb', 'python', 'pymongo'], 'view': 150}
{'_id': ObjectId('6046fb763435c03992181faa'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 100}
{'_id': ObjectId('604701dd3435c03992181fac'), 'author': 'yssim', 'subject': 'MongoDB', 'content': 'SO EASY', 'tags': ['mongodb', 'python', 'pymongo'], 'view': 50}
{'_id': ObjectId('604701de3435c03992181fad'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 100}
{'_id': ObjectId('604704033435c03992181fae'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 'python', 'pymongo'], 'view': 100}
{'_id': ObjectId('6047041b3435c03992181faf'), 'author': 'youngseok', 'subject': 'MongoDataBase', 'content': 'SO DIFFICULT', 'tags': ['mongo', 

In [84]:
# remove
# delete_one()
# delete_many()

articleCollection.delete_many( {'author':'youngseok'})


In [86]:
articleCollection.delete_many( {'author':'yssim'})

In [87]:
for doc in articleCollection.find():
    print(doc)

### scraping - BeautifulSoup , crawling - Selenium

In [91]:
import requests
webpage = requests.get('http://www.daangn.com/hot_articles')
print(webpage.text)

<!DOCTYPE html>
<html lang="ko">
<head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no">
      <link rel="canonical" href="http://www.daangn.com/hot_articles" />

  <title>당근마켓 중고거래 | 당신 근처의 당근마켓</title>
<meta name="description" content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요.">
<meta property="og:url" content="http://www.daangn.com/hot_articles">
<meta property="og:title" content="당근마켓 중고거래 | 당신 근처의 당근마켓">
<meta property="og:description" content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요.">
<meta property="og:site_name" content="당근마켓">
<meta property="og:image" content="http://www.daangn.com/images/meta/home/flea_market.png">
<meta property="og:type" content="article">
<meta property="og:locale" content="ko_KR">
<meta property="fb:app_id" content="1463621440622064">
<meta name="twitter:card" content="summary_l

In [93]:
from bs4 import BeautifulSoup # text로 작업할 수 없기 때문에 --> CSS, Parser등을 이용해 접근해서 사용할 수 있도록 만들어주는 도구

In [95]:
# 이 작업으로 인해 원하는 셀렉터에 접근이 가능해짐 (bs의 객체로 만든 과정)
webpage = requests.get('http://www.daangn.com/hot_articles')
soup    = BeautifulSoup(webpage.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="http://www.daangn.com/hot_articles" rel="canonical"/>
<title>당근마켓 중고거래 | 당신 근처의 당근마켓</title>
<meta content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." name="description"/>
<meta content="http://www.daangn.com/hot_articles" property="og:url"/>
<meta content="당근마켓 중고거래 | 당신 근처의 당근마켓" property="og:title"/>
<meta content="당근마켓에서 거래되는 인기 중고 매물을 소개합니다. 지금 당근마켓에서 거래되고 있는 다양한 매물을 구경해보세요." property="og:description"/>
<meta content="당근마켓" property="og:site_name"/>
<meta content="http://www.daangn.com/images/meta/home/flea_market.png" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<meta content="1463621440622064" property="fb:app_id"/>
<meta content="summary_large_image" name="twitt

In [96]:
print(soup.p)

<p>당근마켓 앱에서 따뜻한 거래를 직접 경험해보세요!</p>


In [97]:
print(soup.p.string)

당근마켓 앱에서 따뜻한 거래를 직접 경험해보세요!
